In [4]:
# %reset -f
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path

dist_to_root = 0    # the distance to project root folder
_project_root = Path.cwd().parents[dist_to_root]
if str(_project_root) not in sys.path:
    sys.path.insert(0, str(_project_root))
    
from utils import bilstm_optmize, load_parameters, data_reshape_generator, create_bilstm_model, data_reshape

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
from sklearn.metrics import r2_score
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import optuna
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

# optuna.logging.set_verbosity(optuna.logging.WARNING)

random_seed = 42
np.random.seed(random_seed)

In [6]:
# load the parameters from file ---------------------------------------------------------
params = load_parameters('parameters.yaml')
# hidden_size = 14

dataset: nasa
labels: ['capacity']
features: ['voltage', 'current', 'temperature', 'relativeTime']
data_groupby: ['cycle']
sequence_length: 50
n_trials: 30


In [7]:
model_dict = {
    5: 'model_nasa/nasa_len5_0_0205.keras',
    10: 'model_nasa/nasa_len10_0_0231.keras',
    20: 'model_nasa/nasa_len20_0_0143.keras',
    30: 'model_nasa/nasa_len30_0_0145.keras',
    50: 'model_nasa/nasa_len50_0_0195.keras'
}

In [8]:
import math
def split_without_overlap_fixed(group: pd.DataFrame, split_size: int):
    length = len(group)
    num_splits = math.floor(length / split_size)  # Calculate number of splits

    splits = []
    start = 0
    for i in range(num_splits):
        end = min(start + split_size, length)
        splits.append(group[start:end])
        start += split_size     # update the start position

    return splits

In [9]:
def generate_test_firstN(test,sequence_length):
    # generate the test data
    first_n_list = []
    for idx, group in test.groupby(params.data_groupby):
        if sequence_length <= len(group):
            group_head = group[:sequence_length]
            first_n_list.append(group_head)
    df = pd.concat(first_n_list,ignore_index=True)
    test_samples, test_targets = data_reshape(df, params.data_groupby, sequence_length, params.features, params.labels)
    return test_samples, test_targets

In [10]:
def generate_test_eachN(test,sequence_length):
    test_samples, test_targets = data_reshape(test, params.data_groupby, sequence_length, params.features, params.labels)
    return test_samples, test_targets

In [11]:
test = pd.read_csv('data_nasa/test_normalized.csv')

for sequence_length in [5,10,20,30,50]:
    print(f'sequence_length = {sequence_length}')
    test_samples, test_targets = generate_test_eachN(test,sequence_length)
    # get the model
    model = load_model(model_dict.get(sequence_length))
    pred = model.predict(test_samples, verbose=0)
    mae = mean_absolute_error(test_targets, pred)
    r2 = r2_score(test_targets, pred)

    print(f"MAE = {mae}, r2 = {r2}")

sequence_length = 5


2025-03-17 17:00:41.012030: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


MAE = 0.008693645583608791, r2 = 0.9948468140224835
sequence_length = 10
MAE = 0.014392305102534723, r2 = 0.9923161875930943
sequence_length = 20
MAE = 0.010254316786441363, r2 = 0.992986232445898
sequence_length = 30
MAE = 0.012859118567827045, r2 = 0.9888864189485923
sequence_length = 50
MAE = 0.01698906852960191, r2 = 0.9843994764961096
